## LangChain Expression Language

For this to work make sure you have latest Langchain installed

In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

### Load Environment Variables

In [2]:
from dotenv import load_dotenv
import os

%load_ext dotenv
%dotenv

In [3]:
openai_api_key  = os.environ['OPANAI_API_KEY']

### Create model

In [4]:
model = ChatOpenAI(openai_api_key=openai_api_key)

### Chat prompt

In [5]:
prompt = ChatPromptTemplate.from_template("Tell me a two sentence story about {event}")

In [6]:
chain = prompt | model

In [7]:
chain.invoke({"event": "first spaceX rocket"})

AIMessage(content='In a historic moment, the first SpaceX rocket soared into the sky, its powerful engines igniting a new era of private space exploration. With bated breath, the world watched as it disappeared into the endless expanse, carrying dreams and aspirations of humanity beyond the confines of Earth.', additional_kwargs={}, example=False)

### Stop Sequence

In [8]:
chain = prompt | model.bind(stop=["\n"])

In [9]:
chain.invoke({"event": "first spaceX rocket"})

AIMessage(content="In a groundbreaking feat, the first SpaceX rocket soared into the sky, carrying the dreams of countless visionaries. As it pierced through the Earth's atmosphere, hope ignited in the hearts of humanity, marking the beginning of a new era in space exploration.", additional_kwargs={}, example=False)

### PromptTemplate + LLM + OutputParser

**StrOutputParser:** This returns a string output instead of the output we are getting above

In [10]:
from langchain.schema.output_parser import StrOutputParser

In [11]:
chain = prompt | model | StrOutputParser()

In [12]:
chain.invoke({"event": "first spaceX rocket"})

"In a historic moment, the first SpaceX rocket soared through the atmosphere, defying gravity's shackles. As it reached the edge of space, humanity's dreams took flight, igniting a new era of space exploration."

### Json Parser

In [13]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [16]:
chain = prompt | model | JsonOutputFunctionsParser()

In [17]:
chain.invoke({"event": "first spaceX rocket"})

OutputParserException: Could not parse function call: 'function_call'

## LLMChain + Retriever

In [18]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough

In [20]:
vectorstore = Chroma.from_texts(["The cats played in the hallway", 
                                 "The cats eat the food I left on the table", 
                                 "John love cats", 
                                 "Joy owns 4 cats"], embedding=OpenAIEmbeddings(openai_api_key=openai_api_key))
retriever = vectorstore.as_retriever()

In [21]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [22]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [23]:
chain.invoke("where the cats eat the food")

'The cats eat the food on the table.'

In [24]:
chain.invoke("where did cats played at?")

'The cats played in the hallway.'

### Multiple Variables

In [36]:
from operator import itemgetter

In [39]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Your tone of voice should be like a: {character}
"""
prompt = ChatPromptTemplate.from_template(template)

In [40]:
chain = {
    "context": itemgetter("question") | retriever, 
    "question": itemgetter("question"), 
    "character": itemgetter("character")
} | prompt | model | StrOutputParser()

In [41]:
chain.invoke({"question": "Where did the cats eat the food from?", "character": "Historian"})

'Based on the available context, it is unclear where exactly the cats ate the food from. The provided documents mention that the cats ate the food, but there is no specific information regarding the location of the food source. Further evidence or details are required to determine the exact place from where the cats consumed their food.'

In [42]:
chain.invoke({"question": "Where did the cats play?", "character": "Student in high school"})

'The cats played in the hallway.'

### Conversational Retriever

In [180]:
from langchain.schema.runnable import RunnableMap, Runnable
from typing import Tuple, List
from langchain.schema import format_document

In [44]:
from langchain.prompts.prompt import PromptTemplate

In [45]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [47]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [49]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

# its better to get the relevant documents and then combine them together, relevant docs in relation to the question asked.
def _combine_documents(docs, document_prompt = DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [61]:
def _format_chat_history(chat_history: List[Tuple]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        human = "Human: " + dialogue_turn[0]
        ai = "Assistant: " + dialogue_turn[1]
        buffer += "\n" + "\n".join([human, ai])
    return buffer

In [63]:
_inputs = RunnableMap(
    {
        "standalone_question": {
            "question": lambda x: x["question"],
            "chat_history": lambda x: _format_chat_history(x['chat_history'])
        } | CONDENSE_QUESTION_PROMPT | ChatOpenAI(temperature=0, openai_api_key=openai_api_key) | StrOutputParser(),
    }
)

In [64]:
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"]
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI(openai_api_key=openai_api_key)

In [67]:
conversational_qa_chain.invoke({
    "question": "Who eat the food on the table?",
    "chat_history": [],
})

AIMessage(content='Based on the context provided, it can be inferred that the cats ate the food that was on the table.', additional_kwargs={}, example=False)

#### How to pass in history

In [69]:
conversational_qa_chain.invoke({
    "question": "Where did the cats play?",
    "chat_history": [("Who eat the food on the table?", "Based on the context provided, it can be inferred that the cats ate the food that was on the table.")],
})

AIMessage(content='The cats played in the hallway.', additional_kwargs={}, example=False)

### Conversation Buffer Memory

In [70]:
from langchain.memory import ConversationBufferMemory

In [72]:
memory = ConversationBufferMemory(return_messages=True, output_key="answer", input_key="question")

In [76]:
memory.load_memory_variables

<bound method ConversationBufferMemory.load_memory_variables of ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key='answer', input_key='question', return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='history')>

In [78]:
memory.load_memory_variables({})

{'history': []}

In [79]:
# First we add a step to load memory
# This needs to be a RunnableMap because its the first input
loaded_memory = RunnableMap(
    {
        "question": itemgetter("question"),
        "memory": memory.load_memory_variables,
    }
)

In [80]:
 # Next we add a step to expand memory into the variables
expanded_memory = {
    "question": itemgetter("question"),
    "chat_history": lambda x: x["memory"]["history"]
}

In [82]:
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x['chat_history'])
    } | CONDENSE_QUESTION_PROMPT | ChatOpenAI(temperature=0, openai_api_key=openai_api_key) | StrOutputParser(),
}

In [83]:
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"]
}

In [84]:
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question")
}

In [86]:
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(openai_api_key=openai_api_key),
    "docs": itemgetter("docs"),
}

In [87]:
# And now we put it all together!
final_chain = loaded_memory | expanded_memory | standalone_question | retrieved_documents | answer

In [90]:
response = final_chain.invoke(input={"question": "Where are the cats playing?"})

In [91]:
response

{'answer': AIMessage(content='Based on the given context, it is not possible to determine where the cats are currently playing.', additional_kwargs={}, example=False),
 'docs': [Document(page_content='The cats played in the hallway', metadata={}),
  Document(page_content='The cats eat the food I left on the table', metadata={}),
  Document(page_content='Joy owns 4 cats', metadata={}),
  Document(page_content='John love cats', metadata={})]}

In [92]:
response = final_chain.invoke(input={"question": "Where are the cats play?"})

In [93]:
response

{'answer': AIMessage(content='The cats play in the hallway.', additional_kwargs={}, example=False),
 'docs': [Document(page_content='The cats played in the hallway', metadata={}),
  Document(page_content='The cats eat the food I left on the table', metadata={}),
  Document(page_content='John love cats', metadata={}),
  Document(page_content='Joy owns 4 cats', metadata={})]}

In [96]:
# Note that the memory does not save automatically
# This will be improved in the future
# For now you need to save it yourself
memory.save_context({"question": "Where are the cats play?"}, {"answer": response["answer"].content})

In [97]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Where are the cats play?', additional_kwargs={}, example=False),
  AIMessage(content='The cats play in the hallway.', additional_kwargs={}, example=False)]}

### Multiple LLM Chains

In [102]:
model = ChatOpenAI(openai_api_key=openai_api_key)

In [99]:
prompt1 = ChatPromptTemplate.from_template("What is the name of the country this {site} is found?")
prompt2 = ChatPromptTemplate.from_template("What is the name of the continent where this {country} is found?") 

In [104]:
chain_one = prompt1 | model | StrOutputParser()

In [106]:
chain_two = {"country": chain_one} | prompt2 | model | StrOutputParser()

In [107]:
chain_two.invoke({"site": "Kilimanjaro"})

'The continent where Mount Kilimanjaro is located in Tanzania is Africa.'

### Using Multiple Variables

In [108]:
prompt1 = ChatPromptTemplate.from_template("What is the name of the country this {site} is found?")
prompt2 = ChatPromptTemplate.from_template("What is the name of the continent where this {country} is found? Respond in {language}") 

In [109]:
chain_one = prompt1 | model | StrOutputParser()

In [110]:
chain_two = {"country": chain_one, "language": itemgetter("language")} | prompt2 | model | StrOutputParser()

In [111]:
chain_two.invoke({"site": "Kilimanjaro", "language": "French"})

'Le mont Kilimandjaro se trouve en Tanzanie, qui est située sur le continent africain.'

In [188]:
prompt1 = ChatPromptTemplate.from_template("generate a random name of a Holiwood actor")
prompt2 = ChatPromptTemplate.from_template("What country is this holiwood actor {actor} from?")
prompt3 = ChatPromptTemplate.from_template("Give me 5 interesting fact about {country}")
prompt4 = ChatPromptTemplate.from_template("Translate {interesting_facts} to French")

chain_one = prompt1 | model | StrOutputParser()
chain_two = prompt2 | model | StrOutputParser()
chain_three = prompt3 | model | StrOutputParser()
chain_four = prompt4 | model | StrOutputParser()

chain_final = RunnableMap(steps={"actor": chain_one}) | RunnableMap(steps={"country": chain_two}) | RunnableMap(steps={"interesting_facts": chain_three}) | chain_four

In [190]:
chain_final.invoke({"language": "French"})
# chain_two.invoke({})

'1. Le manque d\'informations sur un acteur hollywoodien nommé Jack Rivers est assez inhabituel à l\'ère numérique actuelle, où les informations sur les personnalités publiques sont généralement facilement disponibles.\n2. L\'absence de présence ou de reconnaissance significative pour Jack Rivers pourrait suggérer qu\'il pourrait être un acteur relativement inconnu ou émergent qui n\'a pas encore acquis une attention ou une reconnaissance généralisée.\n3. Le mystère entourant Jack Rivers peut susciter la curiosité et l\'intérêt, suscitant des discussions et des théories sur son existence ou son travail potentiel dans l\'industrie du divertissement.\n4. Le manque d\'informations sur Jack Rivers pourrait également indiquer qu\'il a peut-être choisi délibérément de maintenir un profil bas, préférant peut-être une carrière plus privée ou discrète.\n5. Il est également possible que le nom "Jack Rivers" soit un pseudonyme ou un alias utilisé par un acteur pour différentes raisons, telles que

In [192]:
prompt1 = ChatPromptTemplate.from_template("generate a random name of a Holiwood actor")
prompt2 = ChatPromptTemplate.from_template("What country is this holiwood actor {actor} from?")
prompt3 = ChatPromptTemplate.from_template("Give me 5 interesting fact about {country}")
prompt4 = ChatPromptTemplate.from_template("Translate {interesting_facts} to French")

chain_one = prompt1 | model | StrOutputParser()
chain_two = prompt2 | model | StrOutputParser()
chain_three = prompt3 | model | StrOutputParser()
chain_four = prompt4 | model | StrOutputParser()

chain_final = {"actor": chain_one, "country": chain_two, "interesting_facts": chain_three, "language": itemgetter("language")} | prompt4 | model | StrOutputParser()

In [193]:
chain_final.invoke({"language": "French"})

KeyError: 'actor'

### Memory

In [195]:
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnableMap
from langchain.prompts import MessagesPlaceholder

In [197]:
model = ChatOpenAI(openai_api_key=openai_api_key)

In [198]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful chatbot"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [199]:
memory = ConversationBufferMemory(return_messages=True)

In [200]:
memory.load_memory_variables({})

{'history': []}

In [203]:
chain = RunnableMap({
    "input": lambda x: x["input"],
    "memory": memory.load_memory_variables
}) | {
    "input": lambda x: x["input"],
    "history": lambda x: x["memory"]["history"]
} | prompt | model

In [205]:
response = chain.invoke({"input": "Hello there, can you tell me the largest country on Earth"})

In [206]:
response

AIMessage(content='Hello! The largest country on Earth, by land area, is Russia. It spans across both Europe and Asia and covers approximately 17.1 million square kilometers.', additional_kwargs={}, example=False)

### File based memory

In [209]:
from langchain.memory import FileChatMessageHistory

In [211]:
memory_key = f"./test_memory_file.json"

chat_memory = FileChatMessageHistory(file_path=memory_key)
memory = ConversationBufferMemory(
    chat_memory=chat_memory, return_messages=True, output_key="answer", input_key="question")


In [212]:
chain = RunnableMap({
    "input": lambda x: x["input"],
    "memory": memory.load_memory_variables
}) | {
    "input": lambda x: x["input"],
    "history": lambda x: x["memory"]["history"]
} | prompt | model

In [216]:
user_input = {"input": "Hello there, can you tell me the largest country on Earth"}

In [217]:
response = chain.invoke(user_input)

In [218]:
response

AIMessage(content='Hello! The largest country on Earth, by land area, is Russia. It spans across both Europe and Asia, making it transcontinental.', additional_kwargs={}, example=False)

In [219]:
response.content

'Hello! The largest country on Earth, by land area, is Russia. It spans across both Europe and Asia, making it transcontinental.'

###### At the time of creating this, we do not have  auto memory update, we have to do it manually. Here's how you do it. There more you run this the more data gets added in the memory file.

In [221]:
chat_memory.add_user_message(user_input.get("input"))
chat_memory.add_ai_message(response.content)

In [222]:
user_input = {"input": "What was my previous question about?"}

In [223]:
response = chain.invoke(user_input)

In [224]:
response.content

'Your previous question was about the largest country on Earth.'

In [226]:
chat_memory.add_user_message(user_input.get("input"))
chat_memory.add_ai_message(response.content)

### Moderations

Safety guards and violation prevention

In [227]:
from langchain.chains import OpenAIModerationChain
from langchain.llms import OpenAI

In [228]:
model = OpenAI(openai_api_key=openai_api_key)

In [234]:
moderate = OpenAIModerationChain(openai_api_key=openai_api_key)

In [229]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "repeat after me: {input}")
])

#### Without safety guards

In [230]:
chain = prompt | model

In [231]:
chain.invoke({"input": "you are stupid"})

'\n\nYou are stupid.'

#### With moderation

In [235]:
moderated_chain = chain | moderate

In [236]:
moderated_chain.invoke({"input": "you are stupid"})

{'input': '\n\nYou are stupid.',
 'output': "Text was found that violates OpenAI's content policy."}

In [237]:
moderated_chain.invoke({"input": "I love people"})

{'input': '\n\nI love people', 'output': '\n\nI love people'}